# Compare commands for ACIS CTI commands in 2020:145 safe mode recovery

The V1 (legacy) commands archive is used as a reference standard to validate
the new V2 commands archive.

The primary difference is that V1 non-load commands and load stoppages are 
generated using the Chandra.cmd_states non-load commands table and timelines,
while V2 uses only the Flight Command Events from Google Sheets.

In [1]:
import sys
import os
from pathlib import Path
import difflib
import webbrowser

import numpy as np
# Around 2020:145 safe mode
os.environ['KADI_COMMANDS_DEFAULT_STOP'] = '2020:152'

# Using dev versions
sys.path.insert(0, '../')
sys.path.insert(0, str(Path.home() / 'git' / 'parse_cm'))
sys.path.insert(0, str(Path.home() / 'git' / 'testr'))

from kadi.commands import get_cmds, conf

In [2]:
from kadi import logger
# If needed for debugging...
logger.setLevel(1)

### Generate diffs for commands from 2020-01-01 to 2022-01-01

In this era there are real diffs, but all are acceptable. The diffs have been
reviewed individually by aspect team members.

In [3]:
start = '2020:146'
stop = '2020:150'

In [4]:
with conf.set_temp('commands_version', '1'):
    cmds1 = get_cmds(start, stop)

2022-01-07 10:36:20,085 load_idx_cmds: Loaded /Users/aldcroft/ska/data/kadi/cmds.h5 with 1329878 commands


In [5]:
with conf.set_temp('commands_version', '2'):
    cmds2 = get_cmds(start, stop)

2022-01-07 10:36:20,282 update_cmd_events: Getting cmd_events from https://docs.google.com/spreadsheets/d/19d6XqBhWoFjC-z1lS1nM6wLE_zjr4GYB1lOvrEGCbKQ/export?format=csv
2022-01-07 10:36:20,938 update_cmd_events: Writing 23 cmd_events to /Users/aldcroft/.kadi/cmd_events.csv
2022-01-07 10:36:20,947 get_occweb_page: Getting OCCweb FOT/mission_planning/PRODUCTS/APPR_LOADS/2020/MAY with cache=False
2022-01-07 10:36:21,203 get_load_cmds_from_occweb_or_local: Already have /Users/aldcroft/.kadi/loads/MAY0420A.pkl.gz
2022-01-07 10:36:21,210 get_load_cmds_from_occweb_or_local: Already have /Users/aldcroft/.kadi/loads/MAY1120B.pkl.gz
2022-01-07 10:36:21,216 get_load_cmds_from_occweb_or_local: Already have /Users/aldcroft/.kadi/loads/MAY1820A.pkl.gz
2022-01-07 10:36:21,222 get_load_dict_from_cmds: NSM at 2020:145:14:17:20 found for MAY1820A
2022-01-07 10:36:21,223 get_load_cmds_from_occweb_or_local: Already have /Users/aldcroft/.kadi/loads/MAY2420A.pkl.gz
2022-01-07 10:36:21,229 get_load_dict_from

In [6]:
# Commands V1 does not have starcat parameters or LOAD_EVENT commands
for cmds in cmds1, cmds2:
    ok = cmds['type'] == 'MP_STARCAT'
    cmds['params'][ok] = {}
    ok = cmds['type'] == 'LOAD_EVENT'
    cmds.remove_rows(np.where(ok)[0])
    cmds.sort(['date', 'step', 'scs'])

In [7]:
# There are differences in the number of commands
len(cmds1), len(cmds2)

(340, 358)

In [8]:
lines1 = cmds1.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40,
                                     sort_orbit_events=True)
lines2 = cmds2.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40, 
                                     sort_orbit_events=True)

2022-01-07 10:36:21,518 load_pars_dict: Loaded /Users/aldcroft/ska/data/kadi/cmds.pkl with 71769 pars


In [9]:
htmldiffer = difflib.HtmlDiff()
html = htmldiffer.make_file(lines1, lines2, 'kadi commands v1', 'kadi commands v2', context=True)

out = Path('commands_v1_v2_diff.html')
out.write_text(html)

url = 'file://' + str(out.absolute())
webbrowser.open(url)

True